In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from copy import deepcopy
from scipy.io import loadmat
import random
from keras.datasets import mnist
from matplotlib import rcParams
import matplotlib as mpl

In [ ]:
data = loadmat("hw12data.mat")
#The clients hold data
X0 = np.hstack([np.ones([400,1]),data['X'][0:400,:]])
X1 = np.hstack([np.ones([400,1]),data['X'][500:900,:]])
X2 = np.hstack([np.ones([400,1]),data['X'][1000:1400,:]])
X3 = np.hstack([np.ones([400,1]),data['X'][1500:1900,:]])
X4 = np.hstack([np.ones([400,1]),data['X'][2000:2400,:]])
X5 = np.hstack([np.ones([400,1]),data['X'][2500:2900,:]])
X6 = np.hstack([np.ones([400,1]),data['X'][3000:3400,:]])
X7 = np.hstack([np.ones([400,1]),data['X'][3500:3900,:]])
X8 = np.hstack([np.ones([400,1]),data['X'][4000:4400,:]])
X9 = np.hstack([np.ones([400,1]),data['X'][4500:4900,:]])
y0 = np.array(np.zeros([400,1]),dtype = int)
y1 = data['y'][500:900,:]
y2 = data['y'][1000:1400,:]
y3 = data['y'][1500:1900,:]
y4 = data['y'][2000:2400,:]
y5 = data['y'][2500:2900,:]
y6 = data['y'][3000:3400,:]
y7 = data['y'][3500:3900,:]
y8 = data['y'][4000:4400,:]
y9 = data['y'][4500:4900,:]
model0_y0 = 1.0 * (y0==0)
x_test = np.vstack([data['X'][400:500,:],data['X'][900:1000],data['X'][1400:1500],data['X'][1900:2000],data['X'][2400:2500]
              ,data['X'][2900:3000],data['X'][3400:3500],data['X'][3900:4000],data['X'][4400:4500],data['X'][4900:5000]])
X_test = np.hstack([np.zeros([1000,1]),x_test])
y_test = np.vstack([data['y'][400:500,:],data['y'][900:1000],data['y'][1400:1500],data['y'][1900:2000],data['y'][2400:2500]
              ,data['y'][2900:3000],data['y'][3400:3500],data['y'][3900:4000],data['y'][4400:4500],data['y'][4900:5000]])
y_test[y_test == 10] = 0

In [ ]:
def h(X,theta):
    return(1/(1+np.exp(-np.matmul(X,theta))))

def compute_error(X,y,theta):
    return(-(np.sum(y * np.log(h(X,theta)+ 1e-3) + (1-y)* np.log(1-h(X,theta) + 1e-3))))

def FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,modely_2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch):
#The following process is completed on server.
##################################################################################################
    alpha  = 0.001
    theta = np.random.rand(401,S)
    v = np.zeros([401,S])
    minimum_record = []
    failure = 0
##################################################################################################


    for epoch in tqdm(range(0,max_epoch,1)):
        #The following process is completed on server.
        ##################################################################################################
        BCE_old = [0] * S
        ##################################################################################################

        #Send the model to all clients, and clients respond the server with errors.
        #The server do not touch the raw data.
        #The following process is completed on clients.
        ################################################################
        for s in range(0,S,1):
            BCE0 = compute_error(X0,model0_y0,theta[:,[s]])
            BCE1 = compute_error(X1,model0_y1,theta[:,[s]])
            BCE2 = compute_error(X2,model0_y2,theta[:,[s]])
            BCE3 = compute_error(X3,model0_y3,theta[:,[s]])
            BCE4 = compute_error(X4,model0_y4,theta[:,[s]])
            BCE5 = compute_error(X5,model0_y5,theta[:,[s]])
            BCE6 = compute_error(X6,model0_y6,theta[:,[s]])
            BCE7 = compute_error(X7,model0_y7,theta[:,[s]])
            BCE8 = compute_error(X8,model0_y8,theta[:,[s]])
            BCE9 = compute_error(X9,model0_y9,theta[:,[s]])
        ################################################################
        
            BCE_old[s] = BCE0 + BCE1 + BCE2 + BCE3 + BCE4 + BCE5 + BCE6 + BCE7 + BCE8 + BCE9
        minimum_record.append(np.min(BCE_old))

        #Adpative Learning rate
        if epoch > 0:
            if minimum_record[-1] < minimum_record[-2]:
                alpha = alpha * 2
            else:
                failure += 1

            if minimum_record[-1] > minimum_record[-2]:
                break

        if failure == 6:
            failure = 0
            alpha = alpha / 2

        if epoch%1 == 0:
            print(alpha)
            print(minimum_record[-1])

        gbest = deepcopy(theta[:,[np.argmin(BCE_old)]])
        v = deepcopy(alpha * ( w1* v +  w2 * np.random.rand(401,S) * (gbest - theta) + (1-w1 -w2) * (np.random.rand(401,S) * 2-1) ) )
        temp = deepcopy(theta + v)

        BCE_new = [0] * S
        #Send the model to all clients, and clients respond the server with errors.
        #The server do not touch the raw data.
        #The following process is completed on clients.
        ################################################################
        for s in range(0,S,1):
            BCE0 = compute_error(X0,model0_y0,temp[:,[s]])
            BCE1 = compute_error(X1,model0_y1,temp[:,[s]])
            BCE2 = compute_error(X2,model0_y2,temp[:,[s]])
            BCE3 = compute_error(X3,model0_y3,temp[:,[s]])
            BCE4 = compute_error(X4,model0_y4,temp[:,[s]])
            BCE5 = compute_error(X5,model0_y5,temp[:,[s]])
            BCE6 = compute_error(X6,model0_y6,temp[:,[s]])
            BCE7 = compute_error(X7,model0_y7,temp[:,[s]])
            BCE8 = compute_error(X8,model0_y8,temp[:,[s]])
            BCE9 = compute_error(X9,model0_y9,temp[:,[s]])
        ################################################################
            BCE_new[s] = BCE0 + BCE1 + BCE2 + BCE3 + BCE4 + BCE5 + BCE6 + BCE7 + BCE8 + BCE9

        for particles in range(0,S,1):
            if BCE_new[particles]  < BCE_old[particles]:
                theta[:,[particles]] = deepcopy(temp[:,[particles]])
    return(gbest)

# Model 0

In [ ]:
index = 0
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta0 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 1

In [ ]:
index = 1
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta1 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 2

In [ ]:
index = 2
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta2 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 3

In [ ]:
index = 3
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta3 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 4

In [ ]:
index = 4
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta4 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 5

In [ ]:
index = 5
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta5 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 6

In [ ]:
index = 6
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta6 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 7

In [ ]:
index = 7
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta7 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 8

In [ ]:
index = 8
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta8 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Model 9

In [ ]:
index = 9
model_y0 = 1.0 * (y0==index)
model_y1 = 1.0 * (y1==index)
model_y2 = 1.0 * (y2==index)
model_y3 = 1.0 * (y3==index)
model_y4 = 1.0 * (y4==index)
model_y5 = 1.0 * (y5==index)
model_y6 = 1.0 * (y6==index)
model_y7 = 1.0 * (y7==index)
model_y8 = 1.0 * (y8==index)
model_y9 = 1.0 * (y9==index)
w1 = 0
w2 = 0.3
S = 20
max_epoch = 1600
theta9 = FSI_Classification_10(X0,X1,X2,X3,X4,X5,X6,X7,X8,X9,model_y0,model_y1,model_y2,model_y3,model_y4,model_y5,model_y6,
                          model_y7,model_y8,model_y9,w1,w2,S,max_epoch)

# Prediction of the Test Dataset

In [ ]:
total = np.hstack([h(X_test,theta0),h(X_test,theta1),h(X_test,theta2),h(X_test,theta3),h(X_test,theta4),
                   h(X_test,theta5),h(X_test,theta6),h(X_test,theta7),h(X_test,theta8),h(X_test,theta9)])
prediction = np.argmax(total, axis = 1)

In [ ]:
prediction

# Make Confusion Matrix

In [ ]:
confusion_matrix = np.zeros([10,10],dtype = int)
for i in range(0,10,1):
    for j in range(0,10,1):
        confusion_matrix[i,j] = (np.sum(prediction[i * 100:(i+1) * 100] == j))

In [ ]:
confusion_matrix